# Iterative Phase Estimation via the Cloud

This sample code and notebook was written by members of KPMG Quantum team in Australia. It aims to demonstrate expanded capabilities of targets that support adaptive quantum computing, also known as integrated hybrid computing. It makes use of bounded loops, classical function calls, nested conditional if statements, mid-program measurements and qubit reuse.

## Two Dimensional Inner Product Calculation Using Iterative Phase Estimation on Three Qubits

This notebook demonstrates an iterative phase estimation within Q#. It will use iterative phase estimation to calculate an inner product between two 2-dimensional vectors encoded on a target qubit and an ancilla qubit. An additional control qubit is also initialised which will be the only qubit used for measurement.

The circuit begins by encoding the pair of vectors on the target qubit and the ancilla qubit. It then applies an Oracle operator to the entire register, controlled off the control qubit (which is set up in the $\\ket +$ state). The controlled Oracle operator generates a phase on the $\\ket 1$ state of the control qubit. This can then be read by applying a H gate to the control qubit to make the phase observable when measuring.

In [ ]:
# Run this cell first to enable the "%%qsharp" magic command used in later cells
import qsharp

The below cell shows Q# being written directly inside a Jupyter Notebook using the `%%qsharp` 'magic' command

In [ ]:
%%qsharp

// This makes the DumpMachine() function available.
open Microsoft.Quantum.Diagnostics;

operation RandomBit() : Result {
    // Qubits are only accesible for the duration of the scope where they
    // are allocated and are automatically released at the end of the scope.
    use qubit = Qubit();

    // Set the qubit in superposition by applying a Hadamard transformation.
    H(qubit);

    DumpMachine();

    // Measure the qubit. There is a 50% probability of measuring either
    // "Zero" or "One".
    let result = M(qubit);

    Message($"The result of the measurement is {result}.");

    // Reset the qubit so it can be safely released.
    Reset(qubit);
    return result;
}

// Call the operation we defined above.
RandomBit();


As well as invoking Q# operations immediately inline (as the last line in the cell above does), the defined operations can also be invoked from Python

In [ ]:
r = qsharp.eval("RandomBit()")

if r == qsharp.Result.One:
    i = 1
else:
    i = 0

print(f"Python: Got integer {i}!")


In [ ]:
%matplotlib widget

import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

results = qsharp.run("RandomBit()", shots=1000)
# Sort the results so that the histogram labels appear in the correct order
results.sort()
# Count the number of times each result appears
counts = Counter(results)

(values, counts) = counts.keys(), counts.values()
xlabels = np.arange(len(counts))
plt.title("RandomBit() Results")
plt.bar(xlabels, counts)
plt.xticks(xlabels, values)
plt.show()
      

## Submitting jobs to Azure Quantum

Different quantum hardware supports different capabilities, but all Azure Quantum providers support the 'base profile'
as defined in the 'Quantum Intermediate Representation' (QIR) specification. (For more details see <https://aka.ms/qdk.qir>)

To develop code using this base profile, reintialize the Q# compiler, connect to your Azure Quantum workspace, and submit the job.


In [ ]:
# Reset the compiler to target the 'base profile' that all hardware supports

qsharp.init(target_profile=qsharp.TargetProfile.Base)


In [ ]:
# Connect to the Azure Quantum workspace

# If developing locally, on first run this will open a browser to authenticate the
# connection with Azure. In remote scenarios, such as SSH or Codespaces, it may
# be necesssary to install the Azure CLI and run 'az login --use-device-code' to
# authenticate. For unattended scenarios, such as batch jobs, a service principal
# should be configured and used for authentication. For more information, see
# https://learn.microsoft.com/en-us/azure/developer/python/sdk/authentication-overview

import azure.quantum

workspace = azure.quantum.Workspace(
    subscription_id = "MY_SUBSCRIPTION_ID",
    resource_group = "MY_RESOURCE_GROUP",
    name = "MY_WORKSPACE_NAME",
    location = "MY_WORKSPACE_LOCATION",
)


In [ ]:
%%qsharp

operation Random() : Result {
    use q = Qubit();
    H(q);
    let result = M(q);
    Reset(q);
    return result
}

operation RandomNBits(N: Int): Result[] {
    mutable results = [];
    for i in 0 .. N - 1 {
        let r = Random();
        set results += [r];
    }
    return results
}


In [ ]:
# Run the above code for 100 shots against the Rigetti simulator

operation = qsharp.compile("RandomNBits(4)")
target = workspace.get_targets("rigetti.sim.qvm")
job = target.submit(operation, "my-azure-quantum-job", shots=100)

# Wait for the job to complete
job.get_results()
